In [1]:
!pip install Konlpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 68.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 488.6/488.6 kB 32.9 MB/s eta 0:00:00


In [2]:
from konlpy.tag import Komoran
import numpy as np

komoran = Komoran()
text = "오늘 날씨는 구름이 많아요."

# 명사만 추출
nouns = komoran.nouns(text)
print(nouns)

# 단어 사전 구축 및 단어별 인덱스 부여
dics = {}
for word in nouns:
    if word not in dics.keys():
        dics[word] = len(dics)
print(dics)

# 원-핫 인코딩
nb_classes = len(dics)
targets = list(dics.values())
one_hot_targets = np.eye(nb_classes)[targets]
print(one_hot_targets)

['오늘', '날씨', '구름']
{'오늘': 0, '날씨': 1, '구름': 2}
[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]


In [3]:
!pip install gensim

### Word2Vec<임베딩>

In [4]:
from gensim.models import Word2Vec
from konlpy.tag import Komoran
import time


# 네이버 영화 리뷰 데이터 읽어옴
def read_review_data(filename):
    with open(filename, 'r') as f:
        data = [line.split('\t') for line in f.read().splitlines()]
        data = data[1:] # header 제거
    return data


# 측정 시작
start = time.time()

# 리뷰 파일 읽어오기
print('1) 말뭉치 데이터 읽기 시작')
review_data = read_review_data('./ratings.txt')
print(len(review_data)) # 리뷰 데이터 전체 개수
print('1) 말뭉치 데이터 읽기 완료: ', time.time() - start)

# 문장단위로 명사만 추출해 학습 입력 데이터로 만듬
print('2) 형태소에서 명사만 추출 시작')
komoran = Komoran()
docs = [komoran.nouns(sentence[1]) for sentence in review_data]
print('2) 형태소에서 명사만 추출 완료: ', time.time() - start)

# word2vec 모델 학습
print('3) word2vec 모델 학습 시작')
model = Word2Vec(sentences=docs, vector_size=200, window=4, min_count=2, sg=1)
print('3) word2vec 모델 학습 완료: ', time.time() - start)

# 모델 저장
print('4) 학습된 모델 저장 시작')
model.save('nvmc.model')
print('4) 학습된 모델 저장 완료: ', time.time() - start)

# 학습된 말뭉치 개수, 코퍼스 내 전체 단어 개수
print("corpus_count : ", model.corpus_count)
print("corpus_total_words : ", model.corpus_total_words)

1) 말뭉치 데이터 읽기 시작
21750
1) 말뭉치 데이터 읽기 완료:  0.13545632362365723
2) 형태소에서 명사만 추출 시작
2) 형태소에서 명사만 추출 완료:  30.944127321243286
3) word2vec 모델 학습 시작
3) word2vec 모델 학습 완료:  33.89094877243042
4) 학습된 모델 저장 시작
4) 학습된 모델 저장 완료:  33.92615461349487
corpus_count :  21750
corpus_total_words :  112908


In [5]:
print('사랑 : ',model.wv['사랑']) # 200차원의 벡터공간에 분산표현

사랑 :  [ 0.03002603  0.10847727  0.1318822   0.2541617   0.05389578 -0.07656162
  0.13074403  0.05485748 -0.14933805  0.09603977 -0.02551251 -0.27650297
  0.08762668  0.16269487 -0.12790407  0.0296107   0.0566226   0.03670239
  0.04059048 -0.41528758  0.2689289  -0.1973365  -0.00976706  0.07303428
 -0.14303444  0.0283167   0.16082022 -0.03907706 -0.22484617 -0.06709526
  0.15747528  0.12104759  0.20288992 -0.06643476  0.17122811  0.0223372
  0.35942835 -0.01584293 -0.1233888  -0.09566277 -0.25103256 -0.00240293
 -0.43059275  0.13387705  0.26431647 -0.12624565 -0.03152479 -0.149678
  0.07872187  0.05647923 -0.0754791  -0.04129507 -0.02727147 -0.07660388
  0.01438182 -0.19323555 -0.0790622  -0.24797836 -0.44449076 -0.10600182
  0.00446781 -0.06356621  0.05175555 -0.20950884 -0.28184274  0.00514517
  0.11524235  0.20647924 -0.03563675  0.1890517   0.01780554  0.07498461
  0.022695   -0.09273978  0.10426391  0.30145642  0.14643133  0.0064505
 -0.02247307 -0.01161556 -0.05310222 -0.1284925  

In [6]:
from gensim.models import Word2Vec

# 모델 로딩
model = Word2Vec.load('nvmc.model')
print("corpus_total_words : ", model.corpus_total_words)

# '사랑'이란 단어로 생성한 단어 임베딩 벡터
print('사랑 : ', model.wv['사랑'])

# 단어 유사도 계산
print("일요일 = 월요일\t", model.wv.similarity(w1='일요일', w2='월요일'))
print("안성기 = 배우\t", model.wv.similarity(w1='안성기', w2='배우'))
print("대기업 = 삼성\t", model.wv.similarity(w1='대기업', w2='삼성'))
print("일요일 != 삼성\t", model.wv.similarity(w1='일요일', w2='삼성'))
print("히어로 != 삼성\t", model.wv.similarity(w1='히어로', w2='삼성'))

# 가장 유사한 단어 추출
print(model.wv.most_similar("안성기", topn=5))
print(model.wv.most_similar("시리즈", topn=5))

corpus_total_words :  112908
사랑 :  [ 0.03002603  0.10847727  0.1318822   0.2541617   0.05389578 -0.07656162
  0.13074403  0.05485748 -0.14933805  0.09603977 -0.02551251 -0.27650297
  0.08762668  0.16269487 -0.12790407  0.0296107   0.0566226   0.03670239
  0.04059048 -0.41528758  0.2689289  -0.1973365  -0.00976706  0.07303428
 -0.14303444  0.0283167   0.16082022 -0.03907706 -0.22484617 -0.06709526
  0.15747528  0.12104759  0.20288992 -0.06643476  0.17122811  0.0223372
  0.35942835 -0.01584293 -0.1233888  -0.09566277 -0.25103256 -0.00240293
 -0.43059275  0.13387705  0.26431647 -0.12624565 -0.03152479 -0.149678
  0.07872187  0.05647923 -0.0754791  -0.04129507 -0.02727147 -0.07660388
  0.01438182 -0.19323555 -0.0790622  -0.24797836 -0.44449076 -0.10600182
  0.00446781 -0.06356621  0.05175555 -0.20950884 -0.28184274  0.00514517
  0.11524235  0.20647924 -0.03563675  0.1890517   0.01780554  0.07498461
  0.022695   -0.09273978  0.10426391  0.30145642  0.14643133  0.0064505
 -0.02247307 -0.0116

KeyError: "Key '대기업' not present"